<a href="https://colab.research.google.com/github/FrancoPalavicinoG/cellia/blob/main/notebooks/04_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Train, Val & Test Datatsets from Google Drive

Mount Drive

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries

In [29]:
# Core
import pandas as pd
import numpy as np

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Metrics
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, classification_report

# Explainability
import shap
import matplotlib.pyplot as plt
import seaborn as sns

Load Datasets

In [30]:
input_path = "/content/drive/MyDrive/cellia_drive/Datasets/"

Load features

In [31]:
X_train = pd.read_csv(input_path + "X_train.csv")
X_val   = pd.read_csv(input_path + "X_val.csv")
X_test  = pd.read_csv(input_path + "X_test.csv")

Load labels

In [32]:
y_train = pd.read_csv(input_path + "y_train.csv").squeeze()
y_val   = pd.read_csv(input_path + "y_val.csv").squeeze()
y_test  = pd.read_csv(input_path + "y_test.csv").squeeze()

In [33]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(3649, 25) (3649,)
(782, 25) (782,)
(783, 25) (783,)


### Evaluation function

In [34]:
def evaluate_model(model, X, y_true, dataset_name="Validation"):
    y_pred = model.predict(X)
    y_proba = model.predict_proba(X)[:, 1]

    roc_auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print(f"{dataset_name} Results:")
    print(f"ROC-AUC: {roc_auc:.3f} | F1: {f1:.3f} | Precision: {precision:.3f} | Recall: {recall:.3f}")
    print("-" * 60)

    return {"roc_auc": roc_auc, "f1": f1, "precision": precision, "recall": recall}

## Model Training

### Logistic Regression (Baseline)

In [35]:
log_reg = LogisticRegression(
    max_iter=500,          # iteration numbers
    random_state=42,
    class_weight='balanced',  # class balancing
    C=1.0,                 # regularization
    penalty='l2',          # penalty type: 'l1', 'l2', 'elasticnet'
    solver='liblinear'     # solver compatible with the penalty
)


### Random Forest

In [36]:
rf = RandomForestClassifier(
    n_estimators=300,      # number of trees
    max_depth=10,          # maximum depth of each tree
    min_samples_split=2,   # minimum samples to split a node
    min_samples_leaf=1,    # minimum samples per leaf
    max_features='sqrt',   # number of features to consider per split
    random_state=42,
    class_weight='balanced'
)


### XGBoost

In [11]:
# xgb = XGBClassifier(
#     n_estimators=300,
#     learning_rate=0.05,
#     max_depth=6,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     eval_metric='logloss'
# )
# xgb.fit(X_train, y_train)

# evaluate_model(xgb, X_val, y_val, "Validation (XGBoost)")

### LightGBM

In [37]:
lgbm = LGBMClassifier(
    n_estimators=300,      # number of iterations
    max_depth=6,           # maximum tree depth
    num_leaves=31,         # leaves per tree
    learning_rate=0.05,    # learning rate
    min_child_samples=20,  # minimum samples per leaf node
    subsample=0.8,         # fraction of data per tree
    colsample_bytree=0.8,  # fraction of features per tree
    random_state=42,
    scale_pos_weight=1     # class balancing if not using SMOTE
)

### MLP Tabular

In [13]:
# mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
# mlp.fit(X_train, y_train)

# evaluate_model(mlp, X_val, y_val, "Validation (MLP)")

### Train models

In [38]:
models = {"Logistic Regression": log_reg,
          "Random Forest": rf,
          "LightGBM": lgbm}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

Training Logistic Regression...
Validation (Logistic Regression) Results:
ROC-AUC: 0.855 | F1: 0.395 | Precision: 0.264 | Recall: 0.779
------------------------------------------------------------
Training Random Forest...
Validation (Random Forest) Results:
ROC-AUC: 0.829 | F1: 0.414 | Precision: 0.460 | Recall: 0.377
------------------------------------------------------------
Training LightGBM...
[LightGBM] [Info] Number of positive: 359, number of negative: 3290
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000583 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4451
[LightGBM] [Info] Number of data points in the train set: 3649, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.098383 -> initscore=-2.215320
[LightGBM] [Info] Start training from score -2.215320
[LightGBM] [Warning] No further splits with

### Compare models

In [39]:
models = {
    "Logistic Regression": log_reg,
    "Random Forest": rf,
    "LightGBM": lgbm,
}

results = {}
for name, model in models.items():
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)

Validation (Logistic Regression) Results:
ROC-AUC: 0.855 | F1: 0.395 | Precision: 0.264 | Recall: 0.779
------------------------------------------------------------
Validation (Random Forest) Results:
ROC-AUC: 0.829 | F1: 0.414 | Precision: 0.460 | Recall: 0.377
------------------------------------------------------------
Validation (LightGBM) Results:
ROC-AUC: 0.828 | F1: 0.308 | Precision: 0.593 | Recall: 0.208
------------------------------------------------------------


,roc_auc,f1,precision,recall
Logistic Regression,0.854951,0.394737,0.264317,0.779221
Random Forest,0.829290,0.414286,0.460317,0.376623
LightGBM,0.827669,0.307692,0.592593,0.207792


### Hyperparameter Tuning using RandomizedSearchCV

In [42]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from scipy.stats import uniform, randint
import numpy as np

# ---------------------------
# Logistic Regression
# ---------------------------
logreg = LogisticRegression(max_iter=1000, random_state=42)

logreg_param_dist = {
    'C': uniform(0.01, 10),
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

logreg_search = RandomizedSearchCV(
    logreg,
    param_distributions=logreg_param_dist,
    n_iter=20,
    scoring='f1',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

logreg_search.fit(X_train, y_train)
print("Best Logistic Regression params:", logreg_search.best_params_)
print("Best F1:", logreg_search.best_score_)

# ---------------------------
# Random Forest
# ---------------------------
rf = RandomForestClassifier(random_state=42)

rf_param_dist = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(5, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

rf_search = RandomizedSearchCV(
    rf,
    param_distributions=rf_param_dist,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_search.fit(X_train, y_train)
print("Best Random Forest params:", rf_search.best_params_)
print("Best F1:", rf_search.best_score_)

# ---------------------------
# LightGBM
# ---------------------------
lgbm = LGBMClassifier(random_state=42)

lgbm_param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.01, 0.1),
    'n_estimators': randint(200, 1000),
    'min_child_samples': randint(10, 50),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'class_weight': [None, 'balanced']
}

lgbm_search = RandomizedSearchCV(
    lgbm,
    param_distributions=lgbm_param_dist,
    n_iter=10,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

lgbm_search.fit(X_train, y_train)
print("Best LightGBM params:", lgbm_search.best_params_)
print("Best F1:", lgbm_search.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Logistic Regression params: {'C': np.float64(0.5908361216819946), 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}
Best F1: 0.3872623707474291
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Random Forest params: {'class_weight': 'balanced_subsample', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 846}
Best F1: 0.4041519878767852
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[LightGBM] [Info] Number of positive: 359, number of negative: 3290
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000984 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4451
[LightGBM] [Info] Number of data points in the train set: 3649, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] 

## Optimize Hyperparameters Models

### Get the best pre-trained models directly from RandomizedSearchCV.

In [44]:
best_logreg = logreg_search.best_estimator_
best_rf = rf_search.best_estimator_
best_lgbm = lgbm_search.best_estimator_

from sklearn.metrics import classification_report, f1_score

models = {
    "Logistic Regression": best_logreg,
    "Random Forest": best_rf,
    "LightGBM": best_lgbm
}

results = {}
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\n=== {name} ===")
    print("F1 Score:", f1_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    results[name] = evaluate_model(model, X_val, y_val, f"Validation ({name})")

results_df = pd.DataFrame(results).T
results_df.sort_values("roc_auc", ascending=False)


=== Logistic Regression ===
F1 Score: 0.4
              precision    recall  f1-score   support

           0       0.97      0.76      0.85       706
           1       0.27      0.79      0.40        77

    accuracy                           0.77       783
   macro avg       0.62      0.78      0.63       783
weighted avg       0.90      0.77      0.81       783

Validation (Logistic Regression) Results:
ROC-AUC: 0.855 | F1: 0.395 | Precision: 0.264 | Recall: 0.779
------------------------------------------------------------

=== Random Forest ===
F1 Score: 0.4245810055865922
              precision    recall  f1-score   support

           0       0.94      0.91      0.93       706
           1       0.37      0.49      0.42        77

    accuracy                           0.87       783
   macro avg       0.66      0.70      0.68       783
weighted avg       0.89      0.87      0.88       783

Validation (Random Forest) Results:
ROC-AUC: 0.835 | F1: 0.429 | Precision: 0.380 | Re

,roc_auc,f1,precision,recall
Logistic Regression,0.855209,0.394737,0.264317,0.779221
Random Forest,0.834964,0.429379,0.380000,0.493506
LightGBM,0.826711,0.431138,0.400000,0.467532
